# Use a notebook with Kqlmagic to query data in a KQL database 
 
 Use of this notebook is documented in [Microsoft Fabric documentation](https://learn.microsoft.com/fabric/real-time-analytics/jupyter-notebook).

 This flow uses native python packages that are publicly available.

Goal: Query [NYC taxi](https://learn.microsoft.com/azure/open-datasets/dataset-taxi-yellow) that was ingested to KQL DB using Kqlmagic extension. Render a black & white heatmap of taxi pickups in New York City. 

Prerequisites: KQL Database with NYC taxi data loaded.

## High level notebook workflow
- Load dependencies using import commands
- Load the [Kqlmagic](https://pypi.org/project/Kqlmagic/) package to allow connectivity to the KQL database
- Use %kql magic to execute KQL queries on that KQL DB
- Render a black & white heatmap of taxi pickups in New York City

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Import the KQL magic package to enable connectivity to the KQL Database

In [ ]:
!pip install Kqlmagic --no-cache-dir --upgrade

Load the package to memory

In [ ]:
reload_ext Kqlmagic

Set Kqlmagic to automatically return pandas dataframe

In [ ]:
%config Kqlmagic.auto_dataframe = True

 Connect to your Eventhouse query URI and your database

In [ ]:
%kql kusto://code;cluster='enter-database-uri';database='enter-database-name'

This query returns a count of the "trips2" table

In [ ]:
%%kql trips2
| extend 
  pickup_datetime= tpepPickupDateTime
, pickup_latitude = startLat
, pickup_longitude = startLon
| where pickup_datetime between (datetime(2014-01-01)..datetime(2014-01-31))
| where isnotempty(pickup_latitude) and isnotempty(pickup_longitude)
| count

This query sample few records

In [ ]:
%%kql trips2 
| extend 
  pickup_datetime= tpepPickupDateTime
, dropoff_datetime = tpepDropoffDateTime
, pickup_latitude = startLat
, pickup_longitude = startLon
, dropoff_longitude = endLon
, dropoff_latitude = endLat
, vendor_id=vendorID
| where isnotempty(pickup_latitude) and isnotempty(pickup_longitude)
| project vendor_id, pickup_datetime, dropoff_datetime,pickup_longitude, pickup_latitude, dropoff_longitude,dropoff_latitude
| take 3

Define NYC area limits

In [ ]:
south=40.61
north=40.91
west=-74.06
east=-73.77

Aggregate number of pickups to NYC geographical grid

In [ ]:
kql_query = '''
set notruncation;
let South=south; let North=north; let West=west; let East=east; // copy Python variables to ADX
trips2
| extend 
  pickup_datetime= tpepPickupDateTime
, dropoff_datetime = tpepDropoffDateTime
, pickup_latitude = startLat
, pickup_longitude = startLon
, dropoff_longitude = endLon
, dropoff_latitude = endLat
, vendor_id=vendorID
| where pickup_datetime between (datetime(2014-01-01)..datetime(2014-01-31))
| where isnotempty(pickup_latitude) and isnotempty(pickup_longitude)
| extend Lat=round(pickup_latitude, 4), Long=round(pickup_longitude, 4)
| where Lat between(South..North) and Long between(West..East)
| summarize num_pickups=count() by Lat, Long
'''

In [ ]:
%kql aggr_pickups << -query kql_query

In [ ]:
print(aggr_pickups[-4:])

Initialize graphics for the heatmap.

In [ ]:
new_style = {'grid':False}
matplotlib.rc('axes', **new_style)
from matplotlib import rcParams
rcParams['figure.figsize'] = [15, 15]

Draw a map by plotting a heat map over a scatter plot. 

In [ ]:
plt.style.use('dark_background')
p = aggr_pickups.plot(kind='scatter', x='Long', y='Lat', color='white', xlim=(west, east), ylim=(south, north), s=0.02, alpha=0.6)